In [1]:

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd
import cv2
import PIL
import math
from pathlib import Path


In [8]:
device="cpu"
class Config:
    def __init__(self):
        self.epochs = 100
        self.cuda=True
        self.num_classes = 1
        self.batch_size = 4
        self.learning_rate = 0.001
        self.train_size=0.8
        self.dataset = "InterferometerPhoto"
        self.pin_memory = True
        self.momentum = 0.9
        self.step_size = 7
        self.gamma = 0.1
        self.num_workers=0

        #Real photos
        #self.dataset_metadata = "../data/raw/1channel/reference/epsilon.csv"
        #self.data_root_dir = "../data/raw/1channel/photo"

        #Generated unnoised
        self.dataset_metadata = str(Path().resolve().parent)+"\\data\\generated\\unnoised\\reference\\epsilon.csv"
        self.data_root_dir = str(Path().resolve().parent)+"\\data\\generated\\unnoised\photo"

        #Generated noised
        #self.dataset_metadata=str(Path().resolve().parent)+"\\data\\generated\\noised\photo"
        #self.data_root_dir=str(Path().resolve().parent)+"\\data\\generated\\noised\\reference\\epsilon.csv"


        self.data_transforms = transforms.Compose([
                transforms.CenterCrop(448),
                transforms.Resize(224),
                #transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.491, 0.491, 0.491],
                                      std=[0.210, 0.210, 0.210]) 
            ])

class EpsilonDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, annotation_file, transform=None):
        self.root_dir = root_dir
        self.annotations = pd.read_csv(annotation_file,skiprows=0, delim_whitespace=' ')
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img = np.array(cv2.imread(os.path.join(self.root_dir, str(str("%05d" %self.annotations.imgnr[index]))+ ".png"))).astype(np.float32)
        img=PIL.Image.fromarray(np.uint8(img))
        y_label = self.annotations.eps[index]
        if self.transform is not None:
            img = self.transform(img)
        return img, y_label

def prepare_data(config):
    dataset = EpsilonDataset(config.data_root_dir, config.dataset_metadata, transform=config.data_transforms)
    
    g = torch.Generator(device=device).manual_seed(0)
    train_size = int(config.train_size * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size], generator=g)

    print("len(train_dataset):", len(train_dataset),"len(val_dataset):", len(val_dataset))

    loader_params = dict(batch_size=config.batch_size, num_workers=config.num_workers,
                         pin_memory=config.pin_memory, generator=g, shuffle=True)

    train_loader = torch.utils.data.DataLoader(**loader_params, dataset=train_dataset )
    validation_loader = torch.utils.data.DataLoader(**loader_params, dataset=val_dataset )
    
    return {'train': train_loader, 'val': validation_loader}


Load Data

In [9]:
config=Config()

dataloaders = prepare_data(config)
dataset_sizes = {x: len(dataloaders[x]) for x in ['train', 'val']}

train_features, train_labels=next(iter(dataloaders["train"]))

print("Device: ", device)
print("Dataloader train len: ", len(dataloaders["train"]), "val len: ", len(dataloaders["val"]))

len(train_dataset): 96 len(val_dataset): 24
Device:  cpu
Dataloader train len:  24 val len:  6


In [10]:
def customLossFunctionDebug(outputs, labels, totalLoss):
    print("NextOne")
    for i in range (len(outputs)):
        print("i: ", i, "label: ", float(labels[i]), "output:", float(outputs[i]), "diff= ", float(min( abs(abs(labels[i])-abs(outputs[i])) , abs(1-(abs(labels[i])-abs(outputs[i]))) )))

    print("totalLoss:", float(totalLoss))
    return totalLoss

def customLossFunction(outputs, labels):
    totalLoss=0.0
    for i in range (len(outputs)):
        #oneOutputLoss=min( abs(abs(labels[i])-abs(outputs[i])) , abs(1-(abs(labels[i])-abs(outputs[i]))) )
        oneOutputLoss= abs(abs(labels[i])-(outputs[i]))
        totalLoss+=oneOutputLoss
    totalLoss/=len(outputs)
    #customLossFunctionDebug(outputs=outputs, labels=labels, totalLoss=totalLoss)
    return totalLoss

def train_model(model, criterion, optimizer, scheduler, num_epochs):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = -100000
    
    for epoch in range(num_epochs):
        
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
            
            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss+=loss

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            #print("epoch_loss: ", epoch_loss, "running_loss: ", running_loss, "dataset_sizes[phase]: ", dataset_sizes[phase])
            epoch_acc = 1-epoch_loss

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                str(phase), float(epoch_loss), float(epoch_acc)))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(float(best_acc)))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 1)

model_ft = model_ft.to(device)

# Observe that all parameters are being optimized
#optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
optimizer_ft = optim.SGD(model_ft.parameters(), lr=config.learning_rate, momentum=config.momentum)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=config.step_size, gamma=config.gamma)
#print((dataloaders['train'])[0])
model_ft = train_model(model=model_ft, criterion=customLossFunction, optimizer=optimizer_ft, scheduler=exp_lr_scheduler,
                       num_epochs=config.epochs)

Epoch 0/99
train Loss: 0.4202 Acc: 0.5798
val Loss: 0.8136 Acc: 0.1864

Epoch 1/99
train Loss: 0.3775 Acc: 0.6225
val Loss: 0.5667 Acc: 0.4333

Epoch 2/99
train Loss: 0.4329 Acc: 0.5671
val Loss: 0.7664 Acc: 0.2336

Epoch 3/99
train Loss: 0.6275 Acc: 0.3725
val Loss: 0.7254 Acc: 0.2746

Epoch 4/99
train Loss: 0.3875 Acc: 0.6125
val Loss: 0.3632 Acc: 0.6368

Epoch 5/99
train Loss: 0.3600 Acc: 0.6400
val Loss: 0.4789 Acc: 0.5211

Epoch 6/99
train Loss: 0.5108 Acc: 0.4892
val Loss: 0.9622 Acc: 0.0378

Epoch 7/99
train Loss: 0.3551 Acc: 0.6449
val Loss: 0.3032 Acc: 0.6968

Epoch 8/99
train Loss: 0.2407 Acc: 0.7593
val Loss: 0.2773 Acc: 0.7227

Epoch 9/99
train Loss: 0.2240 Acc: 0.7760
val Loss: 0.1465 Acc: 0.8535

Epoch 10/99
train Loss: 0.1790 Acc: 0.8210
val Loss: 0.2606 Acc: 0.7394

Epoch 11/99
train Loss: 0.1671 Acc: 0.8329
val Loss: 0.1843 Acc: 0.8157

Epoch 12/99
train Loss: 0.1919 Acc: 0.8081
val Loss: 0.1995 Acc: 0.8005

Epoch 13/99
train Loss: 0.1737 Acc: 0.8263
val Loss: 0.1473 A

In [ ]:
torch.save(model_ft, 'Conv_GeneratedPhotos_0_9820.pht')

Testowanie

In [40]:
device="cpu"
#model_ft = models.resnet18(pretrained=True)
#num_ftrs = model_ft.fc.in_features
#model_ft.fc = nn.Linear(num_ftrs, 1)
#
#model_ft = model_ft.to(device)
#model_ft.load_state_dict(torch.load('Conv_RealPhotos_0_9745.pht'))
model_ft2=torch.load("Conv_RealPhotos_0_9820.pht")
model_ft2.eval()
dataloaders = prepare_data(config)
device="cuda"
allDiffs=0.0
j=0
for images, labels in dataloaders['val']:
    images, labels = images.cuda(), labels.cuda()
    outputs=model_ft2(images)
    for i in range(len(outputs)):
        diff=abs(float(labels[i]-outputs[i]))
        allDiffs+=diff
        j+=1
        if (j%100==0): print(j, "mean:", allDiffs/j)
        #print("j:", j, "label: ", float(labels[i]), "output: ", float(outputs[i]), "diff=", diff)

print("mean", allDiffs/j)

len(train_dataset): 19200 len(val_dataset): 4800
100 mean: 0.09387498235329986
200 mean: 0.088389149652794
300 mean: 0.08754514856263995
400 mean: 0.08569176271092147
500 mean: 0.08632203799113632
600 mean: 0.08352946497189502
700 mean: 0.0842302260321698
800 mean: 0.08442312653060072
900 mean: 0.08491589876409206
1000 mean: 0.08385394129063935
1100 mean: 0.08194127112881026
1200 mean: 0.08169455604239677
1300 mean: 0.0811609972676692
1400 mean: 0.08042333850171417
1500 mean: 0.0804865208324045
1600 mean: 0.08077163241861854
1700 mean: 0.08046546890948186
1800 mean: 0.07971728358779931
1900 mean: 0.07957782453171125
2000 mean: 0.07951278938865289
2100 mean: 0.07948252423843813
2200 mean: 0.08010959036097946
2300 mean: 0.08022547873587388
2400 mean: 0.0800121398390426
2500 mean: 0.08005703718177974
2600 mean: 0.07963486074935645
2700 mean: 0.07964809501440161
2800 mean: 0.07995607498501028
2900 mean: 0.0799980007882776
3000 mean: 0.08007873030565679
3100 mean: 0.07996379621867691
3200 m

Do przyszłej analizy

In [ ]:
model_conv = torchvision.models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=25)

visualize_model(model_conv)

plt.ioff()
plt.show()